In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os

batch_size = 100
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])
train_data_path = 'horse-or-human/horse-or-human/train'
train_dataset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)  


validation_data_path = 'horse-or-human/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
test_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=0)  

In [2]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import math

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

        self.number_class   = 2

        _size_image     = 100* 100
        _num1           = 50
        _num2           = 50
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, 2, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.ReLU(True))
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        
        self._initialize_weight() 
        
    def _initialize_weight(self):

        for m in self.modules():
            
            n = m.in_features
            m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))

            if m.bias is not None:

                m.bias.data.zero_()

    def forward(self, x):
        #outputs = self.linear(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return outputs

In [4]:
n_iters = 6000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 10000
output_dim = 2
lr_rate = 0.0001

In [5]:
model = LogisticRegression(input_dim, output_dim)

AttributeError: 'LogisticRegression' object has no attribute 'in_features'

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
iter = 0
train_idx=[]
test_idx=[]
train_loss=[]
test_loss=[]
train_ac=[]
test_ac=[]
loss_train = []
accuracy_test   = []
loss_test   = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 100 * 100))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_train_batch    = loss.item() / len(images)
        loss_train.append(loss_train_batch)

        
        train_loss.append(loss.item())
        iter+=1
        train_idx.append(iter)
        
        
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 100*100))
                labels = Variable(labels)
                outputs = model(images)
                loss = criterion(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
                loss_test   += loss.item()
            loss_test = loss_test / len(test_loader.dataset)
            test_loss.append(loss_test)
            accuracy = 100 * float(correct)/total
            test_ac.append(accuracy)
            train_idx.append(iter)
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(train_loss,label = "training loss")
#plt.plot(test_ac,label="test accuracy")
plt.xlabel('iteration')
plt.ylabel('y - axis') 
plt.title('What')
plt.legend() 
plt.show() 

#plt.plot(test_ac,label="test accuracy")
#plt.legend()
#plt.show()